In [1]:
%runfile testcong.py

## Read all optimal curves from the internal database (conductors from $1$ to $400000$) and make a hash table from the $a_p$ for $30$ primes (the first $30$ over $400000$).  This takes about 47 minutes so after doing it once we save the result, and later runs will read this (which takes a few seconds: the saved file is 52M).

In [2]:
try:
    hashtab_13_30 = load('hashtab_13_30')
except IOError:
    hashtab_13_30 = make_hash(13,1,400000,30)
    save(hashtab_13_30, 'hashtab_13_30')

## The hash table is a dictionary: congruent curves will have the same hash (but possibly some incongruent curves will also have the same hash, if they have the same values of $a_p\pmod{13}$ for all $30$ primes tested).  So we extract sets of mutually congruent curves by looking at the dictionary's values (which are lists of labels) of length greater than one.

In [3]:
congruent_sets = [s for s in hashtab_13_30.values() if len(s)>1]
print("The number of nontrivial congruent sets is {}".format(len(congruent_sets)))
from collections import Counter
sizes = Counter()
for s in congruent_sets:
    sizes[len(s)] += 1
print("Distribution of sizes of nontrivial congruent sets: {}".format(sizes))

The number of nontrivial congruent sets is 151
Distribution of sizes of nontrivial congruent sets: Counter({2: 151})


### Test each of these pairs to see if we have an actual congruence, using Kraus's criterion (possibly after twisting to reduce the conductors).  With the default mumax=5000000 there is one pair for which this is not rigorous:  '49a1' and '78400gw1'.

In [4]:
bad_pairs = []
for s in congruent_sets:
    E1 = EllipticCurve(s[0])
    for r in s[1:]:
        E2 = EllipticCurve(r)
        res, info = test_cong(13,E1,E2, mumax=15000000, verbose=False)
        if not res:
            report(res,info,13,s[0],r)
            bad_pairs.append([s[0],r])
if bad_pairs:
    print("{} pairs of curves are not actually 13-congruent: {}".format(len(bad_pairs),bad_pairs))
else:
    print("all {} sets of curves really are 13-congruent".format(len(congruent_sets)))

Curves 49a1 and 78400gw1: testing ell up to 14837760 mod 13
Congruence mod 13 fails for 25921a1 and 78400gw1
 (not even isomorphic up to semisimplication: (29, 2, -2) )
1 pairs of curves are not actually 13-congruent: [['25921a1', '78400gw1']]


### So we have one "false positive" pair which we now remove from the list of congruent pairs:

In [5]:
bad_pair = bad_pairs[0]
assert bad_pair in congruent_sets
congruent_sets.remove(bad_pair)
print("After removing the false positive pair {}, there are {} congruent pairs remaining".format(bad_pair,len(congruent_sets)))

After removing the false positive pair ['25921a1', '78400gw1'], there are 150 congruent pairs remaining


### Now we reduce to a smaller set of pairs such that every congruent pair is a twist of one in the smaller set

In [6]:
j_pairs = [Set([EllipticCurve(s).j_invariant() for s in s2]) for s2 in congruent_sets]
j_sets = list(Set(j_pairs))
print("Out of {} pairs, there are only {} distinct pairs of j-invariants".format(len(j_pairs),len(j_sets)))

Out of 150 pairs, there are only 19 distinct pairs of j-invariants


In [7]:
congruent_pairs = []
for jset in j_sets:
    for s2 in congruent_sets:
        if jset==Set([EllipticCurve(s).j_invariant() for s in s2]):
            congruent_pairs.append(s2)
            break
print("{} congruent pairs up to twist: {}".format(len(congruent_pairs),congruent_pairs))

19 congruent pairs up to twist: [['11025u1', '143325dg1'], ['14157i1', '99099bp1'], ['5175j1', '150075w1'], ['12274j1', '135014j1'], ['208c1', '3952c1'], ['83790dz1', '83790ea1'], ['17640y1', '194040dh1'], ['10192bj1', '173264bo1'], ['20184k1', '20184l1'], ['14175y1', '184275h1'], ['1300b1', '24700j1'], ['1190a1', '265370d1'], ['95370cl1', '95370cm1'], ['1445b1', '10115e1'], ['5070j1', '35490bg1'], ['11638t1', '151294f1'], ['27930cc1', '307230fb1'], ['98010bh1', '98010bi1'], ['314330i1', '314330j1']]


### Test whether all these are irreducible, i.e. have no $13$-isogenies:

In [8]:
for s2 in congruent_pairs:
    for s in s2:
        E = EllipticCurve(s)
        assert not E.isogenies_prime_degree(13)

#### In fact the only isogeny degrees among these curves are 2, 3:

In [9]:
sum([Set(EllipticCurve(s[0]).reducible_primes()) for s in congruent_pairs],Set())

{2, 3}

In [10]:
out = open("mod13pairs_uptotwist.m",'w')
out.write('pairs := [\\\n')
for s in congruent_pairs[:-1]:
    out.write('[EllipticCurve("{}"), EllipticCurve("{}")],\\\n'.format(s[0],s[1]))
s = congruent_pairs[-1]
out.write('[EllipticCurve("{}"), EllipticCurve("{}")]\\\n'.format(s[0],s[1]))
out.write('];\n')
out.close()

In [11]:
len(congruent_pairs)

19

In [12]:
for pair in congruent_pairs:
    sizes = [len(EllipticCurve(lab).isogeny_class()) for lab in pair]
    if sizes != [1,1]:
        print("pair {}: class sizes {}".format(pair,sizes))

pair ['11025u1', '143325dg1']: class sizes [2, 1]
pair ['208c1', '3952c1']: class sizes [2, 1]
pair ['1300b1', '24700j1']: class sizes [2, 1]
pair ['1190a1', '265370d1']: class sizes [2, 1]


In [13]:
[legendre_symbol(a,13) for a in [2,3]]

[-1, 1]

In [14]:
npc=0
out = open("mod13_pairs.m", "w")
out.write('pairs := [\\\n')
for i,pair in enumerate(congruent_pairs):
    cc = [all_c(c) for c in pair]
    for c1 in cc[0]:
        for c2 in cc[1]:
            npc+=1
            out.write('["{}", "{}"]'.format(c1[0],c2[0]))
            if i+1<len(congruent_pairs):
                out.write(',')
            out.write('\\\n')
out.write('];\n')
out.close()           
print("{} pairs written to file".format(npc))            

23 pairs written to file


In [15]:
len(congruent_pairs)

19